### CNN with Image Augmentation (CIFAR10)
images of 10 categories: airplane, automobile, bird, cat, deer, dog, frog, horse, ship and truck. In RGB, 32x32 pixels. The state of the art accuracy is 96.53 percent.


In [1]:
# import tensorflow as tf
import numpy as np                                
# from tensorflow.examples.tutorials.mnist import input_data
import tensorflow.keras as keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Dense, Dropout, Activation,
                                     Flatten)
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

In [2]:
#load data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_rows, img_cols , channels= 32,32,3

In [3]:
# set up image augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
    #zoom_range=0.3
    )
datagen.fit(x_train)

In [4]:
#reshape into images
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, channels)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, channels)
input_shape = (img_rows, img_cols, 1)
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [5]:
#convert integers to float; normalise and center the mean
x_train=x_train.astype("float32")  
x_test=x_test.astype("float32")
mean=np.mean(x_train)
std=np.std(x_train)
x_test=(x_test-mean)/std
x_train=(x_train-mean)/std

In [6]:
# labels
num_classes=10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [7]:
img_size = 32

## 如果include_top = True，shape不能自訂

In [13]:
import tensorflow as tf
import os
num = '0,1'
os.environ["CUDA_VISIBLE_DEVICES"]= num

physical_devices = tf.config.experimental.list_physical_devices('GPU') # 所有你有的GPU
physical_devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [14]:
mirrored_strategy = tf.distribute.MirroredStrategy()

with mirrored_strategy.scope():
    # 定义模型的时候放到镜像策略空间就行
#     model = tf.keras.Sequential([tf.keras.layers.Dense(1, input_shape=(1,))])
    vgg_model = VGG16(weights='imagenet', include_top=False,
                      input_shape=(img_size, img_size, 3))
    x = Flatten()(vgg_model.output)
    x = Dense(128, activation='relu')(x)
    # x = Dropout(0.25)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=vgg_model.inputs, outputs=outputs)

    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])
model.fit(x_train, y_train, batch_size=640, epochs=1, validation_data=(x_test, y_test))

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
Train on 50000 samples, validate on 10000 samples
50000/50000 [==============================] - 15s 295us/sample - loss: 2.3442 - acc: 0.1492 - val_loss: 2.0185 - val_acc: 0.1925


In [15]:
# from tensorflow.keras.utils import multi_gpu_model

# parallel_model = multi_gpu_model(model, gpus=4)
# parallel_model.compile(loss='categorical_crossentropy',
#               optimizer='adam',
#               metrics=['acc'])

# parallel_model.fit(x_train, y_train, batch_size=640, epochs=8, validation_data=(x_test, y_test))

In [16]:
model._name = 'vgg_cifar10'
print(model.name)
model.save(num+'_vgg_cifar10.h5') 

vgg_cifar10


In [17]:
del model